In [1]:
import pandas as pd
import numpy as np
import scipy
import math as mt
import pickle as pkl
import os
import psycopg2
import matplotlib
import matplotlib.pyplot as plt
from numpy.random import normal
import calendar
from scipy.optimize import curve_fit
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,8)
import warnings
warnings.filterwarnings('ignore')
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from datetime import datetime
from datetime import timedelta

In [4]:
# Cargamos los resultados de la regresion poisson para pronosticar ocupacion
data = pd.read_csv("data/arima_pred.csv")
date_start = min(data.dia.values)
date_end = max(data.dia.values)
date_end=datetime.strptime(date_end, '%Y-%m-%d')
date_end= date_end +  timedelta(days=1)
nb_days = (pd.to_datetime(date_end) - pd.to_datetime(date_start)).days
forecasted_demand = data.cn_predic.values
forecasted_demand=pd.DataFrame(forecasted_demand,index=pd.date_range(start=date_start,end=date_end,closed='left'),columns=['occupancy'])
forecasted_demand.occupancy=data.cn_predic.values
forecasted_demand

,occupancy
2018-08-14,133.0
2018-08-15,138.0
2018-08-16,135.0
2018-08-17,113.0
2018-08-18,101.0
2018-08-19,103.0
2018-08-20,121.0
2018-08-21,133.0
2018-08-22,138.0
2018-08-23,135.0


In [5]:
# Let's plot the occupancy forecast using plotly:

occupancy = [go.Scatter(x=forecasted_demand.index, y=forecasted_demand['occupancy'],
                        name='Pronostico de ocupacion para CEINS')]
layout_occ = go.Layout(title='Pronostico de Ocupacion para CEINS',
                       xaxis={'title':'Dia'},
                       yaxis={'title':'Cuartos Noche Ocupados'},
                      
                      )

fig = go.Figure(data=occupancy, layout=layout_occ)
iplot(fig, filename='occ_ts')

In [6]:
# Let's assume a demand price elasticity function:

def demand_price_elasticity(price, nominal_demand, elasticity=-2.0, nominal_price=120.0):
    """Returns demand given a value for the elasticity, nominal demand and nominal price.

    Parameters
    ----------

    price (numpy.ndarray):
        one-dimensional price array. The length of that array should correspond to the
        length of the forecast period.

    nominal_demand (numpy.ndarray):
        one-dimensional forecasted occupancy array. The length of that array should
        correspond to the length of the forecast period.

    elasticity (float):
        value of the elasticity between price and demand. A value of e=-2 is reasonable.

    nominal_price (float):
        room rate for which the forecast was computed.

    Returns
    -------

    A numpy.ndarray of expected demand.
    """

    return nominal_demand * ( price / nominal_price ) ** (elasticity)

In [7]:
import scipy.optimize as optimize

In [8]:
# definition of the objective function:

def objective(p_t, nominal_demand=np.array([50,40,30,20]),
              elasticity=-2.0, nominal_price=1200.0):
    """
    Definition of the objective function. This is the function that want to minimize.
    (minus sign in front)

    Parameters
    ----------

    p_t (numpy.ndarray):
        one-dimensional price array. The length of that array should correspond to the
        length of the forecast period.

    nominal_demand (numpy.ndarray):
        one-dimensional forecasted occupancy array. The length of that array should
        correspond to the length of the forecast period.

    elasticity (float):
        value of the elasticity between price and demand. A value of e=-2 is
        reasonable.

    nominal price (float):
        room rate for which the forecast was computed.

    Returns
    -------

    Value of the objective function (float).

    Note: here we're trying to minimize the objective function. That's where the
    minus sign comes_in.

    """

    return (-1.0 * np.sum( p_t * demand_price_elasticity(p_t, nominal_demand=nominal_demand,
                                                        elasticity=elasticity,
                                                        nominal_price=nominal_price) )) / 100

In [9]:
# Constraints:

def constraint_1(p_t):
    """ This constraint ensures that the prices are positive.
    """
    return p_t


def constraint_2(p_t, capacity=20, forecasted_demand=35.0,
                 elasticity=-2.0, nominal_price=1200.0):
    """ This constraint ensures that the demand does not exceed
    capacity.

    Parameters
    ----------

    p_t (float):
        Room price

    capacity (integer):
        Capacity of the hotel (in rooms).

    forecasted_demand (float):
        Forecasted demand (in rooms) for that night

    elasticity (float):
        slope of the

    nominal_price (float):
        The price for which the forecasted_demand was computed.

    Returns
    -------
    Returns an array of excess capacity.

    """
    return capacity - demand_price_elasticity(p_t, nominal_demand=forecasted_demand,
                                                        elasticity=elasticity,
                                                        nominal_price=nominal_price)

In [26]:
# Let's run the optimization algorithm over four overlapping segments
# of 40.0, 80.0, 120.0, 159.0 room capacity.

capacities = [40.0, 80.0, 120.0, 159.0]

optimization_results = {}
for capacity in capacities:

    # Nominal price associated with forecasted demand:
    nominal_price = 1200.0
    # Forecasted demand:
    nominal_demand = forecasted_demand['occupancy'].values
    # Assumed price elasticity:
    elasticity = -2.0

    # Starting values:
    p_start = 1250.0 * np.ones(len(nominal_demand))

    # bounds on the prices. Let's stick with reasonable values.
    # One could be more sophisticated here and apply constraints
    # that limit the prices to be in range of what competitors
    # are charging, for example.
    bounds = tuple((100.0, 2000.0) for p in p_start)

    # Constraints:
    constraints = ({'type': 'ineq', 'fun':  lambda x:  constraint_1(x)},
               {'type': 'ineq', 'fun':  lambda x, capacity=capacity,
                                           forecasted_demand=nominal_demand,
                                           elasticity=elasticity,
                                           nominal_price=nominal_price: constraint_2(x,capacity=capacity,
                                                                                     forecasted_demand=nominal_demand,
                                                                                     elasticity=elasticity,
                                                                                     nominal_price=nominal_price)})

    opt_results = optimize.minimize(objective, p_start, args=(nominal_demand,
                                                              elasticity,
                                                              nominal_price),
                                    method='SLSQP', bounds=bounds,
                                    constraints=constraints)

    optimization_results[capacity] = opt_results 

In [27]:
# Plotting the resulting rates vs dates.

time_array = np.linspace(1,len(nominal_demand),len(nominal_demand))
rate_df = pd.DataFrame(index=time_array)

for capacity in optimization_results.keys():
    rate_df = pd.concat([rate_df,
                         pd.DataFrame(optimization_results[capacity]['x'],
                                      columns=['{}'.format(capacity)],
                                      index=time_array)],
                        axis=1)

rate_df.index.name = 'Day'
datelist = pd.date_range(start=date_start, end=date_end, closed='left').tolist()
rate_df.index = [ x.date() for x in datelist]

In [39]:
# Generate a pretty table for display purposes.
rate_df = rate_df*1.18
rate_df_to_show = rate_df.copy()

# Renaming the columns:
rate_df_to_show = rate_df_to_show[np.sort(np.asarray(rate_df_to_show.columns))]
rate_df_to_show.columns = ['Capacity left : {}'.format(x) for x in rate_df_to_show.columns]

# Rounding the numbers:
for col in rate_df_to_show.columns:
    rate_df_to_show[col] = rate_df_to_show[col].apply(lambda x: round(x,2))

dow_map = { 6:'Sun', 0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat'}
rate_df_to_show['date'] = rate_df_to_show.index
rate_df_to_show['dow'] = rate_df_to_show['date'].apply(lambda x: dow_map[x.weekday()])
rate_df_to_show['date'] = rate_df_to_show.apply(lambda row: row['dow']+" "+str(row['date']),
                                                axis=1)
rate_df_to_show.index = rate_df_to_show['date'].values
rate_df_to_show.drop(['date','dow'],axis=1,inplace=True)
rate_df_to_show.head(300)


,Capacity left : 120.0,Capacity left : 159.0,Capacity left : 40.0,Capacity left : 80.0
Tue 2018-08-14,1490.73,1295.06,2332.49,1825.76
Wed 2018-08-15,1518.49,1319.18,2360.00,1859.76
Thu 2018-08-16,1501.89,1304.76,2343.64,1839.44
Fri 2018-08-17,1374.08,1193.72,2215.50,1682.90
Sat 2018-08-18,1299.07,1128.56,2132.32,1591.03
Sun 2018-08-19,1311.87,1139.68,2147.08,1606.71
Mon 2018-08-20,1421.89,1235.26,2264.55,1741.45
Tue 2018-08-21,1490.73,1295.06,2332.49,1825.76
Wed 2018-08-22,1518.49,1319.18,2360.00,1859.76
Thu 2018-08-23,1501.89,1304.76,2343.64,1839.44


In [40]:
# Plotting the room rate time series.
# Let's focus on a single week cycle.
price_levels = [go.Scatter(x=rate_df.head(7).index,
                           y=rate_df.head(7)['40.0'],
                           name='Capacity Remaining : 40 rooms'),
                go.Scatter(x=rate_df.head(7).index,
                           y=rate_df.head(7)['80.0'],
                           name='Capacity Remaining : 80 rooms'),
                go.Scatter(x=rate_df.head(7).index,
                           y=rate_df.head(7)['120.0'],
                           name='Capacity Remaining : 120 rooms'),
                go.Scatter(x=rate_df.head(7).index,
                           y=rate_df.head(7)['159.0'],
                           name='Capacity Remaining : 159 rooms')]

layout_prices = go.Layout(title='Rate vs Reservation Date and Current Capacity Levels',
                       xaxis={'title':'Day'}, yaxis={'title':'Rate ($)'})

fig = go.Figure(data=price_levels, layout=layout_prices)
iplot(fig, filename='price_levels_ts')

In [44]:
# Save rate dataframe to local folder:
rate_df = round(rate_df,2)
rate_df.to_csv('data/rates.csv')
rate_df

,40.0,80.0,120.0,159.0
2018-08-14,2332.49,1825.76,1490.73,1295.06
2018-08-15,2360.00,1859.76,1518.49,1319.18
2018-08-16,2343.64,1839.44,1501.89,1304.76
2018-08-17,2215.50,1682.90,1374.08,1193.72
2018-08-18,2132.32,1591.03,1299.07,1128.56
2018-08-19,2147.08,1606.71,1311.87,1139.68
2018-08-20,2264.55,1741.45,1421.89,1235.26
2018-08-21,2332.49,1825.76,1490.73,1295.06
2018-08-22,2360.00,1859.76,1518.49,1319.18
2018-08-23,2343.64,1839.44,1501.89,1304.76


In [ ]:
# deploy a model. Given a date and the capacity for that date,
# returns the rate.
# Reads current capacity from the heroku reservations db
# Read current rates from a csv file stored locally.

def rate_query(arrival_date, departure_date):
    """Given an arrival and a departure dates, this function
    will look up the current number of reservations for those dates
    and return the optimal price according to the rate_df dataframe.

    Parameters
    ----------

    arrival_date (string):
        format YYYY-MM-DD

    departure_date (string):
        format YYYY-MM-DD

    Returns
    -------

    A list containing the room rates for each night.
    """

    # connection to the reservations database.
    # In this case, we used a postgres DB hosted on heroku.
    # The DataScience.com Platform allows you to easily store
    # your access credentials as environment variables. You never
    # have to copy and paste credentials directly in notebook!
    conn = psycopg2.connect(database='my_db',
                       port=os.environ['HOTEL_BOOKINGS_DB_PORT'],
                       password=os.environ['HOTEL_BOOKINGS_DB_PASS'],
                       user=os.environ['HOTEL_BOOKINGS_DB_USER'],
                       host=os.environ['HOTEL_BOOKINGS_DB_HOST'])

    current_bookings = pd.read_sql("SELECT * from bookings where date>=\'{}\' and date <\'{}\' ".format(arrival_date,
                                                                                                        departure_date),conn)
    current_bookings.index = pd.to_datetime(current_bookings['date'])
    current_bookings.drop(['date'], axis=1, inplace=True)

    # Read the rate dataframe:
    rate_df = pd.read_csv('data/rates.csv',header=0, index_col=0)

    # Check for no availability on any of those nights:
    no_avail = current_bookings[current_bookings['rooms_available'] < 1.0 ]
    if len(no_avail) > 0 :
        raise ValueError("No Room available on {}".format(no_avail.index))

    capacity_values = rate_df.columns
    capacity_values = np.sort(capacity_values)[::-1]

    rates = []
    # Look over each date in current_bookings:
    for book_date in current_bookings.index:
        id = 0
        current_capacity = current_bookings.loc[book_date, 'rooms_available']
        while id <= len(capacity_values)-1 and current_capacity <= float(capacity_values[id]):
            tmp = rate_df.loc[str(book_date.date()), capacity_values[id]]
            id+=1
        rates.append(tmp)

    return [ round(rate,2) for rate in rates ]

In [42]:
#rate_df_to_show 
rate_df

,40.0,80.0,120.0,159.0
2018-08-14,2332.490323,1825.761649,1490.728144,1295.061855
2018-08-15,2359.995877,1859.763856,1518.490830,1319.180535
2018-08-16,2343.641742,1839.437958,1501.894803,1304.762828
2018-08-17,2215.500677,1682.896788,1374.079474,1193.723965
2018-08-18,2132.324868,1591.032118,1299.072284,1128.561883
2018-08-19,2147.075638,1606.707690,1311.871335,1139.680988
2018-08-20,2264.546423,1741.449741,1421.887759,1235.257150
2018-08-21,2332.488926,1825.761649,1490.728144,1295.061855
2018-08-22,2360.000000,1859.763856,1518.490830,1319.180535
2018-08-23,2343.639157,1839.437958,1501.894803,1304.762828


In [31]:
rate_df

,40.0,80.0,120.0,159.0
2018-08-14,1976.686715,1547.255635,1263.328936,1097.510046
2018-08-15,1999.996506,1576.071064,1286.856635,1117.949606
2018-08-16,1986.137069,1558.845727,1272.792206,1105.731210
2018-08-17,1877.542947,1426.183719,1164.474130,1011.630479
2018-08-18,1807.054973,1348.332303,1100.908716,956.408376
2018-08-19,1819.555625,1361.616686,1111.755369,965.831346
2018-08-20,1919.107138,1475.804865,1204.989627,1046.828093
2018-08-21,1976.685531,1547.255635,1263.328936,1097.510046
2018-08-22,2000.000000,1576.071064,1286.856635,1117.949606
2018-08-23,1986.134879,1558.845727,1272.792206,1105.731210
